<a href="https://colab.research.google.com/github/JasonLimJS/Grab-AI-for-SEA/blob/master/Model%202%20of%20Ensemble%20kNN%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from scipy import stats
import time
import pickle

from sklearn.externals import joblib

drive.mount('drive')

Create 1- day lagged demand:

In [0]:
raw= pd.read_csv('drive/My Drive/raw.csv',index_col=0)

raw.sort_values(by=['geohash6','time_min','day'],inplace=True)


raw['demand_lag_id']= raw.groupby(['geohash6','time_min']).demand.shift(1)

col_arrangement= ['geohash6', 'day', 'time_min', 'timestamp', 'lat', 'long', 'hour',
                  'minute','demand', 'demand_lag_id']


raw= raw[col_arrangement]

#For those data points which timestamp only has 1 day being observed, its demand is taken as lagged demand  

obs_count= pd.DataFrame(raw.groupby(['geohash6','time_min']).demand.count())
obs_count.reset_index(level=[0,1],drop=False,inplace=True)
obs_count.columns=['geohash6','time_min','demand_count']

raw_combin= pd.merge(left=raw,right=obs_count,left_on=['geohash6','time_min'],right_on=['geohash6','time_min'])
raw_combin['demand_lag_1']= np.where(raw_combin.demand_count==1,raw_combin.demand,raw_combin.demand_lag_id)



arr_sample= ['geohash6', 'day', 'time_min', 'demand_count', 'lat', 'long', 'hour',
             'minute', 'demand_lag_id', 'demand', 'demand_lag_1']

raw_combin=raw_combin[arr_sample]

all_lag_1d= raw_combin[['geohash6','lat','long','time_min','day','demand_lag_1','demand']]
all_lag_1d.sort_values(by=['geohash6','time_min','day'],inplace=True)
all_lag_1d.dropna(inplace=True)

all_lag_1d.to_csv('all_lag_1d.csv')
!cp all_lag_1d.csv drive/My\ Drive/


Model 1: kNN without lag, without PCA

In [0]:
all_lag_1d= pd.read_csv('drive/My Drive/all_lag_1d.csv',index_col=0)

arranged_col= ['lat','long','time_min','demand_lag_1']
X= all_lag_1d[arranged_col].values
y= all_lag_1d.demand.values

X_train, X_test, y_train, y_test= train_test_split(X, y, random_state= 7, \
                                                   test_size=0.20,stratify=X[:,2])


steps=[('scaler',StandardScaler()),('pca',PCA(n_components=3))]

norm_pca= Pipeline(steps)

X_train_scaled= norm_pca.fit_transform(X_train)

norm_mean= norm_pca.named_steps['scaler'].mean_
norm_sd= norm_pca.named_steps['scaler'].var_**0.5
print(norm_mean)
print(norm_sd)

pca_mean= norm_pca.named_steps['pca'].mean_
pca_comp= norm_pca.named_steps['pca'].components_
print(pca_mean)
print(pca_comp)

X_test_scaled= norm_pca.transform(X_test)

Model1= "Model 2"

train_test_kNN(X_train_scaled,X_test_scaled,y_train,y_test, Model1, 60)


######################## Best Model after Hyperparameter Tuning is : #######################

#Training time: 25882.505648851395 secs, R2 value: 0.8242358197645516, RSME: 0.06711758631654619

#KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
#                    metric_params=None, n_jobs=None, n_neighbors=45, p=2,
#                    weights='distance')

# norm_mean= np.array([-5.34740478e+00,9.07638854e+01,6.10487686e+02,1.05574095e-01])
#  norm_std= np.array([5.65621040e-02,1.02704766e-01,3.92263089e+02,1.59564436e-01])
#  pca_mean= np.array([ 6.14163081e-15 ,1.66324146e-14 ,2.60231469e-16,-2.49625107e-14])
#  pca_comp= np.array([[ 0.64256504 ,0.72425171, -0.21969293, -0.11960199],
#                    [-0.21087891,  0.07957761, -0.71483013,  0.66197838],
#                    [ 0.51445685, -0.2009283,   0.46599514,  0.69123838]])



whole_lag_1d_kNN= KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=45, p=2,
                   weights='distance')

whole_lag_1d_kNN.fit(X_train_scaled,y_train)

joblib.dump(whole_lag_1d_kNN,'whole_lag_1d_kNN.pkl')
!cp whole_lag_1d_kNN.pkl drive/My\ Drive/


